# Analyzing CIA Factbook data using SQL

we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

- name — the name of the country.
- area— the country's total area (both land and water).
- area_land — the country's land area in square kilometers.
- area_water — the country's waterarea in square kilometers.
- population — the country's population.
- population_growth— the country's population growth as a percentage.
- birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
- death_rate — the country's death rate, or the number of death per year per 1,000 people.

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [4]:
%%sql
SELECT *
    FROM sqlite_master
  WHERE type='table'

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [6]:
%%sql
SELECT *
    FROM facts
  LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [17]:
%%sql
SELECT MIN(population) as min_pop, MAX(population) as max_pop,
MIN(population_growth) as min_pop_growth, 
MAX(population_growth) as max_pop_growth
    FROM facts

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


In [40]:
%%sql
SELECT name as 'Country w/ min population'
    FROM facts
  WHERE population == (SELECT MIN(population)
                          FROM facts)

(sqlite3.OperationalError) near "as": syntax error
[SQL: SELECT * as 'Country w/ min population'
    FROM facts
  WHERE population == (SELECT MIN(population)
                          FROM facts)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [22]:
%%sql
SELECT name as 'Country w/ max population'
    FROM facts
  WHERE population == (SELECT MAX(population)
                          FROM facts)

Done.


Country w/ max population
World


Removing World and Antarctica from min/max calculation:

In [35]:
%%sql
SELECT MIN(population) as min_pop, MAX(population) as max_pop,
MIN(population_growth) as min_pop_growth, 
MAX(population_growth) as max_pop_growth
    FROM facts
  WHERE name NOT IN ('Antarctica', 'World')

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
48,1367485388,0.0,4.02


In [36]:
%%sql
SELECT ROUND(AVG(population),0) as AVG_populatoin, ROUND(AVG(area),0) as AVG_aera
    FROM facts
  WHERE name NOT IN ('Antarctica', 'World')

Done.


AVG_populatoin,AVG_aera
32377011.0,555094.0


----------
Finding Densely Populated Countries
-------

To identify countries that have both of the following conditons:

- Above-average values for population
- Below-average values for area

In [39]:
%%sql
SELECT name
    FROM facts
  WHERE name NOT IN ('Antarctica', 'World') AND (
        population > (SELECT AVG(population)
                          FROM facts
                        WHERE name NOT IN ('Antarctica', 'World')) 
         AND area < (SELECT AVG(area)
                          FROM facts
                        WHERE name NOT IN ('Antarctica', 'World'))
        )

Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Morocco
Philippines
Poland
Spain


-----
Which country has the most people? Which country has the highest growth rate?
----

In [42]:
%%sql
SELECT name, population, population_growth
    FROM facts
  WHERE population == (SELECT MAX(population)
                          FROM facts
                        WHERE name NOT IN ('Antarctica', 'World')) OR 
        population_growth == (SELECT MAX(population_growth)
                          FROM facts
                        WHERE name NOT IN ('Antarctica', 'World'))

Done.


name,population,population_growth
China,1367485388,0.45
South Sudan,12042910,4.02


-----
Which countries have the highest ratios of water to land? Which countries have more water than land?
-------

In [63]:
%%sql
SELECT name as "Countries w/ max water/land ratio", 
area_land, area_water, round(CAST(area_water as float)/area_land, 2) as water_land_ratio
    FROM facts
  WHERE name NOT IN ('Antarctica', 'World') AND area_water != 0
  ORDER BY water_land_ratio DESC
  LIMIT 10

Done.


Countries w/ max water/land ratio,area_land,area_water,water_land_ratio
British Indian Ocean Territory,60,54340,905.67
Virgin Islands,346,1564,4.52
Puerto Rico,8870,4921,0.55
"Bahamas, The",10010,3870,0.39
Guinea-Bissau,28120,8005,0.28
Malawi,94080,24404,0.26
Netherlands,33893,7650,0.23
Uganda,197100,43938,0.22
Eritrea,101000,16600,0.16
Liberia,96320,15049,0.16


In [50]:
%%sql
SELECT name as "Countries w/ more water than land", 
area_land, area_water, area_water/area_land
    FROM facts
  WHERE area_water > area_land AND name NOT IN ('Antarctica', 'World')

Done.


Countries w/ more water than land,area_land,area_water,area_water/area_land
British Indian Ocean Territory,60,54340,905
Virgin Islands,346,1564,4


------
Which countries will add the most people to their populations next year?
---------

In [56]:
%%sql
SELECT name as "Countries that add most people", 
population, population_growth, 
round(population * population_growth / 100) as added_people
    FROM facts
  WHERE name NOT IN ('Antarctica', 'World')
  ORDER BY added_people DESC
  LIMIT 10

Done.


Countries that add most people,population,population_growth,added_people
India,1251695584,1.22,15270686.0
China,1367485388,0.45,6153684.0
Nigeria,181562056,2.45,4448270.0
Pakistan,199085847,1.46,2906653.0
Ethiopia,99465819,2.89,2874562.0
Bangladesh,168957745,1.6,2703324.0
United States,321368864,0.78,2506677.0
Indonesia,255993674,0.92,2355142.0
"Congo, Democratic Republic of the",79375136,2.45,1944691.0
Philippines,100998376,1.61,1626074.0


-----
Which countries have a higher death rate than birth rate?
-------

In [66]:
%%sql
SELECT name as "Countries w/ death rate > birth rate", 
birth_rate, death_rate, population_growth, 
ROUND(death_rate-birth_rate,2) as 'death - birth rate'
    FROM facts
  WHERE name NOT IN ('Antarctica', 'World') AND death_rate > birth_rate
  ORDER BY death_rate-birth_rate DESC

Done.


Countries w/ death rate > birth rate,birth_rate,death_rate,population_growth,death - birth rate
Bulgaria,8.92,14.44,0.58,5.52
Serbia,9.08,13.66,0.46,4.58
Latvia,10.0,14.31,1.06,4.31
Lithuania,10.1,14.27,1.04,4.17
Ukraine,10.72,14.46,0.6,3.74
Hungary,9.16,12.73,0.22,3.57
Germany,8.47,11.42,0.17,2.95
Slovenia,8.42,11.37,0.26,2.95
Romania,9.14,11.9,0.3,2.76
Croatia,9.45,12.18,0.13,2.73


------
Which countries have the highest population/area ratio?
--------

In [81]:
%%sql
SELECT name as "Countries w/ high pop/area", 
population, area, ROUND(CAST(population as Float) / area) as pop_area
    FROM facts
  WHERE name NOT IN ('Antarctica', 'World') 
  ORDER BY pop_area DESC
  LIMIT 10

Done.


Countries w/ high pop/area,population,area,pop_area
Macau,592731,28,21169.0
Monaco,30535,2,15268.0
Singapore,5674472,697,8141.0
Hong Kong,7141106,1108,6445.0
Gaza Strip,1869055,360,5192.0
Gibraltar,29258,6,4876.0
Bahrain,1346613,760,1772.0
Maldives,393253,298,1320.0
Malta,413965,316,1310.0
Bermuda,70196,54,1300.0
